In [1]:
# Configuration file for the LLM project
import pandas as pd
import numpy as np
from youtube_transcript_api import YouTubeTranscriptApi
import re
import sys
sys.path.append('../src')

from food_buddy_api import get_food_buddy_recommendations

Functions

In [2]:
def clean_transcript_text(transcript):
    """Rranscript cleaner adding commas after each snippet"""
    
    # Add dot after each snippet, then join
    text_parts = []
    for snippet in transcript.snippets:
        clean_text = snippet.text.strip()
        if clean_text:
            text_parts.append(clean_text + ",")
    
    text = " ".join(text_parts)
    
    # Clean up
    text = re.sub(r'\[\xa0__\xa0\]', '', text)  # Remove censored parts
    text = re.sub(r'\[Music\]', '', text)       # Remove music tags
    text = re.sub(r'\s+', ' ', text).strip()    # Fix spacing
    
    # Capitalize first letter
    if text:
        text = text[0].upper() + text[1:]
    
    return text

In [3]:
# Query test
query = "pasta with chicken"
recipe = get_food_buddy_recommendations(query)
print(recipe)

[{'Name': 'Chicken Dijon & Pasta', 'Image_first': None, 'TotalTime_str': '1 hour 5 minutes', 'recipe_instructions_clean': 'Preheat oven to 400 degrees. Place chicken in shallow roasting pan. Mix mustard and dressing. Spoon half of mixture over chicken. Bake, uncovered, 40 minutes. Combine soup, 1/2 c water and remaining mustard mixture. Toss pasta wirth sauce, 2/3 c French Fried Onions, vegetables and parsley. Spoon mixture around chicken. Bake, uncovered, 15 minutes or until chicken is no longer pink in center. Sprinkle with remaining 2/3 c onions. Bake 1 minute or until onions are golden.', 'ingredients_clean': 'chicken, mustard, pasta, onion, tomatoes, zucchini, parsley', 'Calories': 567.7, 'similarity': 0.6422552343553112}, {'Name': 'Easy Lemon Pasta With Chicken', 'Image_first': None, 'TotalTime_str': '40 minutes', 'recipe_instructions_clean': 'Cook the pasta in a large pot of boiling salted water, until al dente. Drain well. Season chicken with salt and pepper. Heat a large grill

In [4]:
# Transcript test from YouTube video
video_id = "mdqb3fVqZgM"
api = YouTubeTranscriptApi()
transcript = api.fetch(video_id)

print(transcript)

FetchedTranscript(snippets=[FetchedTranscriptSnippet(text="i'm gonna go [\xa0__\xa0] ballistic get a", start=0.08, duration=5.6), FetchedTranscriptSnippet(text='grip oh dear raj', start=2.32, duration=4.479), FetchedTranscriptSnippet(text='come here you', start=5.68, duration=2.56), FetchedTranscriptSnippet(text="it's raw", start=6.799, duration=5.681), FetchedTranscriptSnippet(text="it's [\xa0__\xa0] loud it's weird okay", start=8.24, duration=9.16), FetchedTranscriptSnippet(text='come on man what is that', start=12.48, duration=4.92), FetchedTranscriptSnippet(text='what what what what', start=17.76, duration=4.16), FetchedTranscriptSnippet(text='what is that', start=19.76, duration=5.04), FetchedTranscriptSnippet(text="ah it's like barbie's vomit it's like", start=21.92, duration=5.76), FetchedTranscriptSnippet(text="purple snot i swear to god it's [\xa0__\xa0]", start=24.8, duration=4.639), FetchedTranscriptSnippet(text="pinker inside that paris hilton's", start=27.68, duration=4.16

In [5]:
# Clean transcript test
transcript_clean = clean_transcript_text(transcript)
print(transcript_clean)

I'm gonna go ballistic get a, grip oh dear raj, come here you, it's raw, it's loud it's weird okay, come on man what is that, what what what what, what is that, ah it's like barbie's vomit it's like, purple snot i swear to god it's , pinker inside that paris hilton's, lipstick, just touch that fish there, just touch how dry it's got more , wrinkles on it than i have and i'm 44, years of age that should be , cooked fresh, thanks to there hurry up dear, barney yeah but i'm not dick face chef, yeah say that again i said i'm not dick, face you're pissed are you that last, bucket look at me look at me guys not as, pissed as i am you ah, donkey no you're cooking like a donkey i, didn't realize they shrunk down that, much chef, it's a bit like your brain not only is, it shrunk but it disappears, all of you look it's not it's just no, it's not about all it's not about that, oh, damn it pal a bit exploded all over my, face and like i have little tiny like, bits of halibut in my eyes that's what

`